In [103]:
import json
import pandas as pd
import numpy as np

sekcje = pd.read_csv("sekcje_format.csv", sep=";", header=0)

id_name = dict(zip(list(sekcje["id_sekcji"]), list(sekcje["nazwa_sekcji"])))
name_id = dict(zip(list(sekcje["nazwa_sekcji"]), list(sekcje["id_sekcji"])))

with open("headers.json", "r") as f:
    headers = json.load(f)

In [104]:
# If sections_present key is a list with elements of depth both A.1, and A.1.1, choose the one with more dots
for i in range(len(headers["header"])):
    if len(headers["sections_present"][i])>1:
        deepest = max([len(x.split(".")) for x in headers["sections_present"][i]])
        headers["sections_present"][i] = [x for x in headers["sections_present"][i] if len(x.split("."))==deepest]

In [105]:
# Group by config (size, font, color) and get how many different sections are present for that config
config_dict = {}
for i in range(len(headers["header"])):
    config = (headers["size"][i], headers["color"][i], headers["font"][i])
    if config not in config_dict:
        config_dict[config] = set()
    config_dict[config].update(headers["sections_present"][i])

config_dict = {k: len(v) for k, v in config_dict.items()}

In [106]:
df = pd.DataFrame(headers)
df["sections_present"] = df["sections_present"].apply(lambda x: x[0])
df["tuple"] = df.apply(lambda x: (x["size"], x["color"], x["font"]), axis=1)
df["num_ocs"] = df["tuple"].map(config_dict)
df['nazwa_sekcji'] = df['sections_present'].map(id_name).combine_first(df['header'])
df['id_sekcji_tmp'] = df['sections_present'].map(name_id).combine_first(df['sections_present']).apply(lambda x: x.replace(" ", "").strip())
df['sekcja_nadrzedna'] = df["id_sekcji_tmp"].apply(lambda x: ".".join(x.split(".")[:-1])).replace({"A": "A.", "B":"B.", "C": "C.", "D": "D.", "E": "E."})
df['id_sekcji_nadrzednej2'] = df.merge(sekcje, how="left", left_on="sekcja_nadrzedna", right_on="id_sekcji")["id"]
df['id_sekcji'] = df.merge(sekcje, how="left", left_on="id_sekcji_tmp", right_on="id_sekcji")["id"]
df['nazwa_sekcji'] = df['id_sekcji_tmp'] + " "+df['nazwa_sekcji']


def remove_duplicate_first_word(text):
    words = text.strip().split()
    if len(words) > 1 and words[0] == words[1]:
        return ' '.join(words[1:])
    else:
        return text

df['nazwa_sekcji'] = df['nazwa_sekcji'].apply(remove_duplicate_first_word)
def skip_first_n_chars(row):
    n = len(row['nazwa_sekcji'].split())
    return " ".join(row['text'].split()[max(0,n-1):])
df['tresc']=df.apply(skip_first_n_chars, axis=1)
df['id_sekcji'] = df['id_sekcji'].fillna(df['id_sekcji_tmp']+"_DEL")
df = df.sort_values(by=["id_sekcji", "num_ocs"], ascending=False)
df = df.drop_duplicates(subset=["id_sekcji"], keep="first")
df['id_sekcji'] = df['id_sekcji'].apply(lambda x: np.nan if isinstance(x, str) and "_DEL" in x else x)
df.loc[:, ['id_sekcji_nadrzednej2', 'id_sekcji', 'nazwa_sekcji', 'tresc']].reset_index(drop=True)
# df.loc[:, ['id_sekcji_nadrzednej2', 'id_sekcji', 'nazwa_sekcji', 'tresc']].reset_index(drop=True).to_csv("sekcje.csv", index=False)

,id_sekcji_nadrzednej2,id_sekcji,nazwa_sekcji,tresc
0,NaN,NaN,Podsumowanie do sprawozdania o wypłacalności i...,finansowej
1,28.0,NaN,D.3.9 D.3.9. Zobowiązania z tytułu świadczeń p...,"Wszystkie programy określonych świadczeń, któr..."
2,28.0,NaN,D.3.8 D.3.8. Pozostałe zobowiązania (niewykaza...,Pozycje zobowiązań niewykazywane w żadnej z w/...
3,28.0,NaN,"D.3.7 D.3.7. Pozostałe zobowiązania (handlowe,...",Niniejsza pozycja zawiera wartość wszystkich z...
4,28.0,NaN,D.3.6 D.3.6. Zobowiązania z tytułu reasekuracji,Zgodnie z przyjętymi przez Towarzystwo zasadam...
5,28.0,NaN,D.3.5 D.3.5. Zobowiązania z tytułu ubezpieczeń...,Niniejsza pozycja wykazywana jest w wartości g...
6,28.0,NaN,D.3.4 D.3.4. Zobowiązania finansowe inne niż z...,W bilansie ekonomicznym w pozycji Zobowiązania...
7,28.0,NaN,D.3.3 D.3.3. Rezerwy z tytułu odroczonego poda...,Sposób wyceny i prezentacji odroczonych podatk...
8,28.0,NaN,D.3.2 D.3.2. Pozostałe rezerwy (inne niż techn...,W Towarzystwie wycena pozostałych rezerw odbyw...
9,28.0,NaN,D.3.1 D.3.1. Zobowiązania warunkowe,"Zobowiązania warunkowe, jeśli są istotne są uz..."


In [107]:
df.loc[:, ['id_sekcji_nadrzednej2', 'id_sekcji', 'nazwa_sekcji', 'tresc']].reset_index(drop=True)

,id_sekcji_nadrzednej2,id_sekcji,nazwa_sekcji,tresc
0,NaN,NaN,Podsumowanie do sprawozdania o wypłacalności i...,finansowej
1,28.0,NaN,D.3.9 D.3.9. Zobowiązania z tytułu świadczeń p...,"Wszystkie programy określonych świadczeń, któr..."
2,28.0,NaN,D.3.8 D.3.8. Pozostałe zobowiązania (niewykaza...,Pozycje zobowiązań niewykazywane w żadnej z w/...
3,28.0,NaN,"D.3.7 D.3.7. Pozostałe zobowiązania (handlowe,...",Niniejsza pozycja zawiera wartość wszystkich z...
4,28.0,NaN,D.3.6 D.3.6. Zobowiązania z tytułu reasekuracji,Zgodnie z przyjętymi przez Towarzystwo zasadam...
5,28.0,NaN,D.3.5 D.3.5. Zobowiązania z tytułu ubezpieczeń...,Niniejsza pozycja wykazywana jest w wartości g...
6,28.0,NaN,D.3.4 D.3.4. Zobowiązania finansowe inne niż z...,W bilansie ekonomicznym w pozycji Zobowiązania...
7,28.0,NaN,D.3.3 D.3.3. Rezerwy z tytułu odroczonego poda...,Sposób wyceny i prezentacji odroczonych podatk...
8,28.0,NaN,D.3.2 D.3.2. Pozostałe rezerwy (inne niż techn...,W Towarzystwie wycena pozostałych rezerw odbyw...
9,28.0,NaN,D.3.1 D.3.1. Zobowiązania warunkowe,"Zobowiązania warunkowe, jeśli są istotne są uz..."
